In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd 'drive/MyDrive/aut_sem3/embedding_pointclouds_test1/self_rag_test1'

/content/drive/MyDrive/aut_sem3/embedding_pointclouds_test1/self_rag_test1


In [3]:
!pip install chromadb
!pip install langchain-community langchain-core
!pip install sentence-transformers
!pip install langchain_mistralai
!pip install tiktoken
!pip install huggingface_hub
!pip install "datasets>=2.6.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 23.3 MB/s eta 0:00

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [5]:
import chromadb
from chromadb.utils import embedding_functions

# Create the open-source embedding function
embedding_function1 = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2") #Same results can be seen using the "hkunlp/instructor-xl" embedding

client = chromadb.PersistentClient(path="./chroma_db2")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

#Step 1: Loading the Dataset

In [ ]:
# # # col = client.get_or_create_collection(name="triviaqa-rc", embedding_function=embedding_function1)
#client.delete_collection("musique")
# # #col1 = client.get_or_create_collection(name="triviaqa", embedding_function=embedding_function1)
col1 = client.get_or_create_collection(name="musique", embedding_function=embedding_function1)

In [ ]:
from datasets import load_dataset

ds = load_dataset("dgslibisey/MuSiQue")

Generating train split:   0%|          | 0/19938 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2417 [00:00<?, ? examples/s]

In [ ]:
# Access the training dataset
train_dataset = ds['train']
validation_dataset = ds['validation']

# Print the first record of the training dataset
print(train_dataset[0])



{'id': '2hop__482757_12019', 'paragraphs': [{'idx': 0, 'title': 'Pakistan Super League', 'paragraph_text': 'Pakistan Super League (Urdu: پاکستان سپر لیگ \u202c \u200e; PSL) is a Twenty20 cricket league, founded in Lahore on 9 September 2015 with five teams and now comprises six teams. Instead of operating as an association of independently owned teams, the league is a single entity in which each franchise is owned and controlled by investors.', 'is_supporting': False}, {'idx': 1, 'title': 'Serena Wilson', 'paragraph_text': 'Serena Wilson (August 8, 1933 – June 17, 2007), often known just as "Serena", was a well-known dancer, choreographer, and teacher who helped popularize belly dance in the United States. Serena\'s work also helped legitimize the dance form and helped it to be perceived as more than burlesque or stripping. Serena danced in clubs in her younger years, opened her own studio, hosted her own television show, founded her own dance troupe, and was the author of several book

In [ ]:
vd=validation_dataset.select(range(200))
vd

Dataset({
    features: ['id', 'paragraphs', 'question', 'question_decomposition', 'answer', 'answer_aliases', 'answerable'],
    num_rows: 200
})

In [ ]:
statement_text1 = []

for i, record in enumerate(vd):
  # Extract only the values from the record and append them as a single list
  value = record['answer']
  statement_text1 .append(value)

print(statement_text1[1])


Mike Medavoy


In [ ]:
statement_text2= []

for i, record in enumerate(vd):
  # Extract only the values from the record and append them as a single list
  values = record['question']
  statement_text2.append(values)

print(statement_text2[1])


Who founded the company that distributed the film UHF?


In [ ]:
all_context=[]
for i, record in enumerate(vd):
  for j in range(0, len(record['paragraphs'])):
    value = record['paragraphs'][j]['paragraph_text']
    search_results = value
    all_context.append(search_results)
    #print(search_results)

#Step 2: Vectorize the Context

In [ ]:
from tqdm import tqdm

# Prepare documents and metadata
total_records = len(vd)  # Total number of records
progress_bar = tqdm(total=total_records, desc="Uploading documents to DB", unit="record")

# Prepare documents and metadata
doc_id = 0  # Initialize document ID counter
for i, record in enumerate(vd):
  for j in range(0, len(record['paragraphs'])):
    value = record['paragraphs'][j]['paragraph_text']
    search_results = value

    # Add the entire document as a single entry with unique ID
    col1.add(
        ids=[f"{doc_id}"],  # Unique identifier for the document
        documents=[search_results],
    )

    doc_id += 1  # Increment document ID counter

  progress_bar.update(1)  # Update progress bar for each record processed

progress_bar.close()  # Close the progress bar when done


In [ ]:
#Exampple of Data
col1.query(query_texts=["Randall County"], n_results=10, include=['documents'])

{'ids': [['959',
   '704',
   '2982',
   '1144',
   '2864',
   '396',
   '561',
   '1153',
   '1959',
   '1947']],
 'distances': None,
 'metadatas': None,
 'embeddings': None,
 'documents': [["New Richland is a city in Waseca County, Minnesota, United States. The city was founded in 1877. The population was 1,203 at the 2010 census. Located in rural south-central Minnesota, New Richland is a small town with a strong agricultural base. In 2013, the NRHEG High School Women's Basketball team won the State Tournament.",
   'Culberson County-Allamoore Independent School District is a public school district based in Van Horn, Texas (USA). The district serves all of Culberson County and eastern portions of Hudspeth County. The district was created in 1995 by the consolidation of the Culberson County and Allamore districts.',
   'This is a list of villages in Møre og Romsdal, a county of Norway. The list excludes cities located in Møre og Romsdal.',
   'McFarland School District is located in 

In [6]:
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate

#1.Musique Dataset Evaluation on Llama3.1:7b

In [7]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 9.9 MB/s eta 0:00:00


In [9]:
%xterm
#Enter the following cmds on the terminal:
#1. `curl -fsSL https://ollama.com/install.sh | sh`
#2. `ollama serve & ollama pull llama3.1:latest`

Launching Xterm...

<IPython.core.display.Javascript object>

In [12]:
# Import Ollama module from Langchain
from langchain_community.llms import Ollama

# Initialize an instance of the Ollama model
llm = Ollama(model="llama3.1:latest")

# Invoke the model to generate responses
response = llm.invoke("Who are you?", temperature=0)
print(response)

I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


In [ ]:
def generate_sub_query(previous_contexts, previous_sub_queries, original_query):
    # Format previous contexts and sub-questions into a history string
    history = ""
    for i, context in enumerate(previous_contexts):
        history += f"Context {i+1}: {context}\n"
    for i, sub_query in enumerate(previous_sub_queries):
        history += f"Sub-Query {i+1}: {sub_query}\n"

    # Generate a concise sub-question focusing on important keywords
    prompt = (f"Given the following history of contexts and sub-questions:\n{history}\n"
              f"Generate a short and focused sub-question to help answer the original question: '{original_query}'. "
              f"Ensure the sub-question is concise and includes the most relevant keywords. "
              f"Do not repeat any previous sub-questions or similar queries. Please only give the sub-question. No chitchats.")
    response = llm.invoke(prompt, temperature=0)

    sub_query = response.strip()

    # Ensure the new sub-query is not a repetition of previous ones
    if sub_query in previous_sub_queries:
        sub_query = f"Fallback: What key details can you provide about '{original_query}'?"
        sub_query = llm.invoke(sub_query, temperature=0).strip()

    return sub_query



def query_database(sub_query, previous_contexts, n_results=5):
    # Query the database with the generated sub-query
    retrieved_docs = col1.query(query_texts=[sub_query], n_results=n_results, include=['documents'])

    # Flatten the list of documents (as ChromaDB might return them as a list of lists)
    flattened_docs = [item for sublist in retrieved_docs['documents'] for item in sublist]

    # Filter out duplicate contexts that have already been retrieved
    unique_contexts = [doc for doc in flattened_docs if doc not in previous_contexts]

    return unique_contexts[:1]  # Return only the first unique context


def generate_final_answer(all_contexts, previous_sub_queries, original_query):
    # Combine all contexts into a single string
    combined_context = " ".join(all_contexts)

    # Format previous sub-questions and combine them with the contexts
    history = ""
    for i, (context, sub_query) in enumerate(zip(all_contexts, previous_sub_queries)):
        history += f"Context {i+1}: {context}\nSub-Query {i+1}: {sub_query}\n"

    # Generate the final answer considering all contexts and sub-queries
    prompt = (f"Given the following history of contexts and sub-questions:\n{history}\n"
              f"and considering the original question: '{original_query}', "
              f"provide a precise and direct answer that synthesizes the information from all contexts. "
              f"Ensure that the answer addresses the specific details asked in the original question. "
              f"Please provide a direct answer. No chitchats.")

    response = llm.invoke(prompt, temperature=0)

    return response



def cascaded_rag(original_query, max_iterations=5):
    all_contexts = []
    previous_sub_queries = []

    for _ in range(max_iterations):
        # Generate a concise and focused sub-question based on the most recent context or the original question
        sub_query = generate_sub_query(all_contexts, previous_sub_queries, original_query)
        print(f"Generated Sub-Query: {sub_query}")

        # Query the database with the sub-question and filter duplicates
        retrieved_docs = query_database(sub_query, all_contexts)
        if not retrieved_docs:
            print("No more relevant unique documents found.")
            break

        # Collect the retrieved context
        context = " ".join(retrieved_docs)
        all_contexts.append(context)

        print(f"Retrieved Context: {context}")

        # Update the list of previous sub-queries
        previous_sub_queries.append(sub_query)

    # Generate the final answer considering all contexts and sub-queries
    final_answer = generate_final_answer(all_contexts, previous_sub_queries, original_query)

    return final_answer

# Example original query
original_query = "What is the seat of the county where Van Hook Township is located?"
final_answer = cascaded_rag(original_query)

print(f"Final Answer: {final_answer}")


Generated Sub-Query: What is the name of the county in which Van Hook Township is situated?
Retrieved Context: Van Hook Township is a township in Mountrail County in the U.S. state of North Dakota. At the time of the 2000 Census, its population was 42, and estimated to be 41 as of 2009.
Generated Sub-Query: What is the name of Mountrail County's administrative center?
Retrieved Context: The Mountrail County Courthouse in Stanley, North Dakota was built in 1914 and served Mountrail County as its courthouse continuously since then. It was designed by architects Buechner & Orth in what can be termed Academic Revival architecture and/or Late 19th and 20th Century Revivals architecture. It was listed on the National Register of Historic Places in 1978; a area was included in the listing.
Generated Sub-Query: What is the administrative center of Mountrail County?
Retrieved Context: The Mount Meigs Campus is a juvenile corrections facility of the Alabama Department of Youth Services located i

In [ ]:
def generate_sub_query(previous_contexts, previous_sub_queries, original_query):
    # Format previous contexts and sub-questions into a history string
    history = ""
    for i, context in enumerate(previous_contexts):
        history += f"Context {i+1}: {context}\n"
    for i, sub_query in enumerate(previous_sub_queries):
        history += f"Sub-Query {i+1}: {sub_query}\n"

    # Generate a concise sub-question focusing on important keywords
    prompt = (f"Given the following history of contexts and sub-questions:\n{history}\n"
              f"Generate a short and focused sub-question to help answer the original question: '{original_query}'. "
              f"Ensure the sub-question is concise and includes the most relevant keywords. "
              f"Do not repeat any previous sub-questions or similar queries. Please only give the sub-question. No chitchats.")
    response = llm.invoke(prompt, temperature=0)

    sub_query = response.strip()

    # Ensure the new sub-query is not a repetition of previous ones
    if sub_query in previous_sub_queries:
        sub_query = f"Fallback: What key details can you provide about '{original_query}'?"
        sub_query = llm.invoke(sub_query, temperature=0).strip()

    return sub_query

def query_database(sub_query, previous_contexts, n_results=5):
    # Query the database with the generated sub-query
    retrieved_docs = col1.query(query_texts=[sub_query], n_results=n_results, include=['documents'])

    # Flatten the list of documents (as ChromaDB might return them as a list of lists)
    flattened_docs = [item for sublist in retrieved_docs['documents'] for item in sublist]

    # Filter out duplicate contexts that have already been retrieved
    unique_contexts = [doc for doc in flattened_docs if doc not in previous_contexts]

    return unique_contexts[:1]  # Return only the first unique context

def generate_final_answer(all_contexts, previous_sub_queries, original_query):
    # Combine all contexts into a single string
    combined_context = " ".join(all_contexts)

    # Format previous sub-questions and combine them with the contexts
    history = ""
    for i, (context, sub_query) in enumerate(zip(all_contexts, previous_sub_queries)):
        history += f"Context {i+1}: {context}\nSub-Query {i+1}: {sub_query}\n"

    # Generate the final answer considering all contexts and sub-queries
    prompt = (f"Given the following history of contexts and sub-questions:\n{history}\n"
              f"and considering the original question: '{original_query}', "
              f"provide a precise and direct answer that synthesizes the information from all contexts. "
              f"Ensure that the answer addresses the specific details asked in the original question. "
              f"Please provide a direct answer. No chitchats.")

    response = llm.invoke(prompt, temperature=0)

    return response.strip()

def cascaded_rag(original_query, max_iterations=5):
    all_contexts = []
    previous_sub_queries = []

    for _ in range(max_iterations):
        # Generate a concise and focused sub-question
        sub_query = generate_sub_query(all_contexts, previous_sub_queries, original_query)
        print(f"Generated Sub-Query: {sub_query}")

        # Query the database with the sub-question and filter duplicates
        retrieved_docs = query_database(sub_query, all_contexts)
        if not retrieved_docs:
            print("No more relevant unique documents found.")
            break

        # Collect the retrieved context
        context = " ".join(retrieved_docs)
        all_contexts.append(context)

        print(f"Retrieved Context: {context}")

        # Update the list of previous sub-queries
        previous_sub_queries.append(sub_query)

    # Generate the final answer considering all contexts and sub-queries
    final_answer = generate_final_answer(all_contexts, previous_sub_queries, original_query)

    return final_answer.strip()

# Example original query
original_query = "Who is the lead singer of the band performing Bang Bang Rock & Roll?"
final_answer = cascaded_rag(original_query)

print(f"Final Answer: {final_answer}")


Generated Sub-Query: What is the name of the vocalist on the song "Bang Bang Rock & Roll"?
Retrieved Context: Bang Bang Rock & Roll is the debut studio album by British rock band Art Brut. It was re-released in 2006 with bonus CD.
Generated Sub-Query: What is Eddie Argos' role in Art Brut?
Retrieved Context: Eddie Argos (born Kevin Macklin 25 October 1979) is the lead singer of English rock band Art Brut, and writer of comics.
Generated Sub-Query: What is Eddie Argos' role in Art Brut's music?
Retrieved Context: Golgoth 13 were a French band, formed in 2000 in Bayonne (France) by Sébastien Lasserre (guitar) and Grégory Sentenac (organ). They were soon joined by Sylvain Aubert on scratches and sampling, and they records their first album "Golgoth 13 et leur "fantastic rodéo sound"" in August 2000. They were later joined by Hervé Zwingelstein as a fourth member, on bass guitar. Following the recording of the band's second album "Bruce Lee died in Galway" in 2004, Grégory Sentenac is repl

In [ ]:
def evaluate_questions(statement_text2, statement_text1):
  for i, question in enumerate(statement_text2):
    print(f"Processing Question {i+1}: {question}")

    # Generate the final answer using the cascaded_rag function
    generated_answer = cascaded_rag(question)

    # Retrieve the ground truth answer
    ground_truth = statement_text1[i]

    # Print the question, generated answer, and ground truth answer
    print(f"Generated Answer: {generated_answer}")
    print(f"Ground Truth Answer: {ground_truth}")
    print("="*50)  # Separator for readability

# Run the evaluation
evaluate_questions(statement_text2[:101], statement_text1[:101])

Processing Question 1: Who is the spouse of the Green performer?
Generated Sub-Query: What is the name of the person married to the lead singer of the band Green?
Retrieved Context: Green is the fourth studio album by British progressive rock musician Steve Hillage. Written in spring 1977 at the same time as his previous album, the funk-inflected "Motivation Radio" (1977), "Green" was originally going to be released as "The Green Album" as a companion to "The Red Album" (the originally intended name for "Motivation Radio"). However, this plan was dropped and after a US tour in late 1977, "Green" was recorded alone, primarily in Dorking, Surrey, and in London.
Generated Sub-Query: What is the marital status and partner of Steve Hillage, the musician behind the album "Green"?
Retrieved Context: Miquette Giraudy (born 9 February 1953, Nice, France) is a keyboard player and vocalist, best known for her work in Gong and with her partner Steve Hillage. She and Hillage currently form the core

#2. Musique Dataset Evaluation on Llama2:7b

In [10]:
%xterm
#Enter the following cmds on the terminal:
#1. `curl -fsSL https://ollama.com/install.sh | sh`
#2. `ollama serve & ollama pull llama2:7b`

Launching Xterm...

<IPython.core.display.Javascript object>

In [14]:
# Import Ollama module from Langchain
from langchain_community.llms import Ollama

# Initialize an instance of the Ollama model
llm = Ollama(model="llama2:7b")

# Invoke the model to generate responses
response = llm.invoke("Who are you?", temperature=0)
print(response)


I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I am trained on a massive dataset of text from the internet and can generate human-like responses to a wide range of topics and questions. I can be used to create chatbots, virtual assistants, and other applications that require natural language understanding and generation capabilities.


In [ ]:
def generate_sub_query(previous_contexts, previous_sub_queries, original_query):
    # Format previous contexts and sub-questions into a history string
    history = ""
    for i, context in enumerate(previous_contexts):
        history += f"Context {i+1}: {context}\n"
    for i, sub_query in enumerate(previous_sub_queries):
        history += f"Sub-Query {i+1}: {sub_query}\n"

    # Generate a concise sub-question focusing on important keywords
    prompt = (f"Based on the following history of contexts and sub-questions:\n{history}\n"
              f"and considering the original question: '{original_query}', "
              f"generate a precise sub-question that targets specific gaps in information from the contexts. "
              f"Ensure the sub-question is actionable, focuses on key details, and is phrased in a way that makes it effective for searching in a database. "
              f"Please avoid broad or lengthy queries, and do not include phrases like 'Sub-Question:' or similar in your response. "
              f"Provide only the sub-question.")

    response = llm.invoke(prompt, temperature=0)

    sub_query = response.strip()

    # Ensure the new sub-query is not a repetition of previous ones
    if sub_query in previous_sub_queries or not sub_query:
        sub_query = f"What specific details are needed to answer '{original_query}'?"
        sub_query = llm.invoke(sub_query, temperature=0).strip()

    return sub_query

def query_database(sub_query, previous_contexts, n_results=5):
    # Query the database with the generated sub-query
    retrieved_docs = col1.query(query_texts=[sub_query], n_results=n_results, include=['documents'])

    # Flatten the list of documents (as ChromaDB might return them as a list of lists)
    flattened_docs = [item for sublist in retrieved_docs['documents'] for item in sublist]

    # Filter out duplicate contexts that have already been retrieved
    unique_contexts = [doc for doc in flattened_docs if doc not in previous_contexts]

    return unique_contexts[:1]  # Return only the first unique context

def generate_final_answer(all_contexts, previous_sub_queries, original_query):
    # Combine all contexts into a single string
    combined_context = " ".join(all_contexts)

    # Format previous sub-questions and combine them with the contexts
    history = ""
    for i, (context, sub_query) in enumerate(zip(all_contexts, previous_sub_queries)):
        history += f"Context {i+1}: {context}\nSub-Query {i+1}: {sub_query}\n"

    # Generate the final answer considering all contexts and sub-queries
    prompt = (f"Given the following history of contexts and sub-questions:\n{history}\n"
              f"and considering the original question: '{original_query}', "
              f"provide a precise and direct answer that synthesizes the information from all contexts. "
              f"Make sure the answer directly addresses the specific details asked in the original question.")

    response = llm.invoke(prompt, temperature=0)

    return response.strip()

def cascaded_rag(original_query, max_iterations=5):
    all_contexts = []
    previous_sub_queries = []

    for _ in range(max_iterations):
        # Generate a concise and focused sub-question
        sub_query = generate_sub_query(all_contexts, previous_sub_queries, original_query)
        print(f"Generated Sub-Query: {sub_query}")

        # Query the database with the sub-question and filter duplicates
        retrieved_docs = query_database(sub_query, all_contexts)
        if not retrieved_docs:
            print("No more relevant unique documents found.")
            break

        # Collect the retrieved context
        context = " ".join(retrieved_docs)
        all_contexts.append(context)

        print(f"Retrieved Context: {context}")

        # Update the list of previous sub-queries
        previous_sub_queries.append(sub_query)

    # Generate the final answer considering all contexts and sub-queries
    final_answer = generate_final_answer(all_contexts, previous_sub_queries, original_query)

    return final_answer.strip()

# Example original query
original_query = "Who is the lead singer of the band performing Bang Bang Rock & Roll?"
final_answer = cascaded_rag(original_query)

print(f"Final Answer: {final_answer}")


Generated Sub-Query: Sub-question: What are the lead singer's name and date of birth?
Retrieved Context: Eddie Argos (born Kevin Macklin 25 October 1979) is the lead singer of English rock band Art Brut, and writer of comics.
Generated Sub-Query: Eddie Argos' name and date of birth are: Eddie Argos (born Kevin Macklin, October 25, 1979).
Retrieved Context: Morris Mike Medavoy (born January 21, 1941) is an American film producer and executive, co-founder of Orion Pictures (1978), former chairman of TriStar Pictures, former head of production for United Artists (1974–1978) and current chairman and CEO of Phoenix Pictures.
Generated Sub-Query: What is Eddie Argos' date of birth?
Retrieved Context: Richard of Shrewsbury, Duke of York KG (born 17 August 1473), was the sixth child and second son of King Edward IV of England and Elizabeth Woodville, born in Shrewsbury. Richard and his older brother, who briefly reigned as King Edward V of England, mysteriously disappeared shortly after Richar

In [ ]:
def evaluate_questions(statement_text2, statement_text1):
  for i, question in enumerate(statement_text2):
    print(f"Processing Question {i+1}: {question}")

    # Generate the final answer using the cascaded_rag function
    generated_answer = cascaded_rag(question)

    # Retrieve the ground truth answer
    ground_truth = statement_text1[i]

    # Print the question, generated answer, and ground truth answer
    print(f"Generated Answer: {generated_answer}")
    print(f"Ground Truth Answer: {ground_truth}")
    print("="*50)  # Separator for readability

# Run the evaluation
evaluate_questions(statement_text2[:101], statement_text1[:101])

Processing Question 1: Who is the spouse of the Green performer?
Generated Sub-Query: Sub-question: Which Green performer are you referring to?
Retrieved Context: The collaborative show features Williams, Keith Moseley (of The String Cheese Incident) and Jeff Austin (of Yonder Mountain String Band) performing bluegrass versions of Grateful Dead songs. The group performed under the name Grateful Grass.
Generated Sub-Query: Which Green performer are you referring to?
Retrieved Context: Green is the fourth studio album by British progressive rock musician Steve Hillage. Written in spring 1977 at the same time as his previous album, the funk-inflected "Motivation Radio" (1977), "Green" was originally going to be released as "The Green Album" as a companion to "The Red Album" (the originally intended name for "Motivation Radio"). However, this plan was dropped and after a US tour in late 1977, "Green" was recorded alone, primarily in Dorking, Surrey, and in London.
Generated Sub-Query: Whic

#3. Musique Dataset Evaluation on Mistral7b

In [15]:
%xterm
#Enter the following cmds on the terminal:
#1. `curl -fsSL https://ollama.com/install.sh | sh`
#2. `ollama serve & ollama pull mistral:7b`

Launching Xterm...

<IPython.core.display.Javascript object>

In [16]:
# Import Ollama module from Langchain
from langchain_community.llms import Ollama

# Initialize an instance of the Ollama model
llm = Ollama(model="mistral:7b")

# Invoke the model to generate responses
response = llm.invoke("Who are you?", temperature=0)
print(response)

 I am a Large Language Model trained by Mistral AI.


In [ ]:
def generate_sub_query(previous_contexts, previous_sub_queries, original_query):
    # Format previous contexts and sub-questions into a history string
    history = ""
    for i, context in enumerate(previous_contexts):
        history += f"Context {i+1}: {context}\n"
    for i, sub_query in enumerate(previous_sub_queries):
        history += f"Sub-Query {i+1}: {sub_query}\n"

    # Generate a concise sub-question focusing on important keywords
    prompt = (f"Given the following history of contexts and sub-questions:\n{history}\n"
              f"Generate a short and focused sub-question to help answer the original question: '{original_query}'. "
              f"Ensure the sub-question is concise and includes the most relevant keywords. "
              f"Do not repeat any previous sub-questions or similar queries. Please only give the sub-question. No chitchats.")
    response = llm.invoke(prompt, temperature=0)

    sub_query = response.strip()

    # Ensure the new sub-query is not a repetition of previous ones
    if sub_query in previous_sub_queries:
        sub_query = f"Fallback: What key details can you provide about '{original_query}'?"
        sub_query = llm.invoke(sub_query, temperature=0).strip()

    return sub_query

def query_database(sub_query, previous_contexts, n_results=5):
    # Query the database with the generated sub-query
    retrieved_docs = col1.query(query_texts=[sub_query], n_results=n_results, include=['documents'])

    # Flatten the list of documents (as ChromaDB might return them as a list of lists)
    flattened_docs = [item for sublist in retrieved_docs['documents'] for item in sublist]

    # Filter out duplicate contexts that have already been retrieved
    unique_contexts = [doc for doc in flattened_docs if doc not in previous_contexts]

    return unique_contexts[:1]  # Return only the first unique context

def generate_final_answer(all_contexts, previous_sub_queries, original_query):
    # Combine all contexts into a single string
    combined_context = " ".join(all_contexts)

    # Format previous sub-questions and combine them with the contexts
    history = ""
    for i, (context, sub_query) in enumerate(zip(all_contexts, previous_sub_queries)):
        history += f"Context {i+1}: {context}\nSub-Query {i+1}: {sub_query}\n"

    # Generate the final answer considering all contexts and sub-queries
    prompt = (f"Given the following history of contexts and sub-questions:\n{history}\n"
              f"and considering the original question: '{original_query}', "
              f"provide a precise and direct answer that synthesizes the information from all contexts. "
              f"Ensure that the answer addresses the specific details asked in the original question. "
              f"Please provide a direct answer. No chitchats.")

    response = llm.invoke(prompt, temperature=0)

    return response.strip()

def cascaded_rag(original_query, max_iterations=5):
    all_contexts = []
    previous_sub_queries = []

    for _ in range(max_iterations):
        # Generate a concise and focused sub-question
        sub_query = generate_sub_query(all_contexts, previous_sub_queries, original_query)
        print(f"Generated Sub-Query: {sub_query}")

        # Query the database with the sub-question and filter duplicates
        retrieved_docs = query_database(sub_query, all_contexts)
        if not retrieved_docs:
            print("No more relevant unique documents found.")
            break

        # Collect the retrieved context
        context = " ".join(retrieved_docs)
        all_contexts.append(context)

        print(f"Retrieved Context: {context}")

        # Update the list of previous sub-queries
        previous_sub_queries.append(sub_query)

    # Generate the final answer considering all contexts and sub-queries
    final_answer = generate_final_answer(all_contexts, previous_sub_queries, original_query)

    return final_answer.strip()

# Example original query
original_query = "Who is the lead singer of the band performing Bang Bang Rock & Roll?"
final_answer = cascaded_rag(original_query)

print(f"Final Answer: {final_answer}")


Generated Sub-Query: "What is the name of the lead singer in Bang Bang Rock & Roll band?"
Retrieved Context: Bang Bang Rock & Roll is the debut studio album by British rock band Art Brut. It was re-released in 2006 with bonus CD.
Generated Sub-Query: "Who is the frontman of Art Brut, the band that released Bang Bang Rock & Roll?"
Retrieved Context: Eddie Argos (born Kevin Macklin 25 October 1979) is the lead singer of English rock band Art Brut, and writer of comics.
Generated Sub-Query: "Who is the frontman of the band that performed Bang Bang Rock & Roll?"
Retrieved Context: Burney and Chandler soon formed the Folkadots, while Wiseman formed Gary & the Wiseman. Burney and Chandler, along with Morrill, also formed the band Slaw. Around this time, Reddick formed the band Terminal Seasons. Not too long after, Jaret Reddick and Lance Morrill formed coolfork!, which Chris Burney later joined. The band was in full swing by 1993, playing such venues as the Refuge. A few months later, after 

In [ ]:
def evaluate_questions(statement_text2, statement_text1):
  for i, question in enumerate(statement_text2):
    print(f"Processing Question {i+1}: {question}")

    # Generate the final answer using the cascaded_rag function
    generated_answer = cascaded_rag(question)

    # Retrieve the ground truth answer
    ground_truth = statement_text1[i]

    # Print the question, generated answer, and ground truth answer
    print(f"Generated Answer: {generated_answer}")
    print(f"Ground Truth Answer: {ground_truth}")
    print("="*50)  # Separator for readability

# Run the evaluation
evaluate_questions(statement_text2[:101], statement_text1[:101])

Processing Question 1: Who is the spouse of the Green performer?
Generated Sub-Query: "What is the name of the partner of the Green Artist?"
Retrieved Context: Green is the fourth studio album by British progressive rock musician Steve Hillage. Written in spring 1977 at the same time as his previous album, the funk-inflected "Motivation Radio" (1977), "Green" was originally going to be released as "The Green Album" as a companion to "The Red Album" (the originally intended name for "Motivation Radio"). However, this plan was dropped and after a US tour in late 1977, "Green" was recorded alone, primarily in Dorking, Surrey, and in London.
Generated Sub-Query: "Who is the partner/spouse of Steve Hillage?"
Retrieved Context: "McMillan & Wife" revolved around a 40-ish San Francisco police commissioner, Stuart McMillan (Rock Hudson) and his attractive, bright, affable wife Sally (Susan Saint James), who was in her twenties. Often, the storylines featured Mac and Sally attending fashionable 